In [1]:
import numpy as np 
from joblib import Parallel, delayed
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import open3d as o3d
from scipy.optimize import minimize
from scipy.linalg import lstsq
import os

pcd_list = np.load("xyz_files.npy")
normals_list = np.load("normals_files.npy")

pcds=[]
norms = []
for pcd, normals in zip(pcd_list, normals_list):
    pcds.append(np.loadtxt(pcd))
    norms.append(np.loadtxt(normals))
pcds = np.array(pcds)
norms = np.array(norms)
print(pcds.shape, norms.shape)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
(381, 100000, 3) (381, 100000, 3)


In [7]:
init_norms =[]
def find_neighs(points, k=250):
    nbrs = NearestNeighbors(n_neighbors=k, algorithm= 'kd_tree').fit(points)
    distances, indices = nbrs.kneighbors(points) 
    return distances, indices

def pca_plane(points):
    # print(points.shape)
    pca = PCA(n_components=3)
    pca.fit(points)
    return pca.components_[2]/np.linalg.norm(pca.components_[2])

def process_pcd(pcd):
    _, indices = find_neighs(pcd, 250)
    neighs = pcd[indices]
    norm_esti_pca = Parallel(n_jobs=-1)(delayed(pca_plane)(point_neighs) for point_neighs in neighs)
    return np.array(norm_esti_pca)
    

for pcd in pcds:
    init_norms.append(process_pcd(pcd))
    
# np.save("init_norms.npy", np.array(init_norms))


In [8]:
init_norms[0].shape

(100000, 3)

In [4]:

dists, indices = find_neighs(pcds[0], 250)
# print(pcd[indices].shape)
neighs = pcds[0][indices]
print(neighs.shape)
# norm_esti_pca = Parallel(n_jobs=-1)(delayed(pca_plane)(point_neighs) for point_neighs in neighs)

# norm_= normals/np.linalg.norm(normals[:300,:], axis=1)[:300,np.newaxis]
# dot_pca = np.diag(np.dot(norm_esti_pca[:300,:] , norm_.T)).mean()

# print(dot_pca, dot_ls, dot_min)

(100000, 250, 3)


In [2]:
import numpy as np 
init_norms = np.load("init_normals.npy")
print(init_norms.shape)
ori_norms_paths = np.load("normals_files.npy")
ori_norms = []
for ori_norms_path in ori_norms_paths:
    ori_norms.append(np.loadtxt(ori_norms_path))
ori_norms = np.array(ori_norms)
print(ori_norms.shape)

(381, 100000, 3)
(381, 100000, 3)


In [6]:
xyz_paths = np.load("xyz_files.npy")
xyzs = []
for xyz_path in xyz_paths:
    xyzs.append(np.loadtxt(xyz_path))
xyzs = np.array(xyzs)
print(xyzs.shape)
# np.save("xyzs.npy", xyzs)

(381, 100000, 3)


In [5]:
# np.save("origianl_normals.npy", ori_norms)

In [3]:
import torch
# ori_norms = torch.from_numpy(ori_norms)
# init_norms = torch.from_numpy(init_norms)
sum=0
for i in range(ori_norms.shape[0]):
    dot = np.absolute(np.diag(np.dot(ori_norms[i][:300,:], init_norms[i][:300,:].T)))
    sum+=dot.mean()
print(sum/ori_norms.shape[0])

0.9291552780560308


In [29]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyzs[232])
pcd.normals = o3d.utility.Vector3dVector(init_norms[232])
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

In [ ]:
import os
os._exit(00)

: 